In [ ]:
import numpy as np
import pandas as pd

# 1. Helper function to calculate entropy
def entropy(dataset):
    # Count the occurrences of each class label in the dataset
    class_counts = dataset.iloc[:, -1].value_counts()
    # Calculate the entropy of the dataset using the formula
    prob = class_counts / len(dataset)
    return -np.sum(prob * np.log2(prob))

# 2. Function to calculate information gain
def information_gain(dataset, feature):
    # Calculate the entropy of the original dataset
    total_entropy = entropy(dataset)

    # Get the values of the feature and their frequencies
    feature_values = dataset[feature].value_counts()
    weighted_entropy = 0

    # For each feature value, calculate the weighted entropy of the split
    for value, count in feature_values.items():
        subset = dataset[dataset[feature] == value]
        weighted_entropy += (count / len(dataset)) * entropy(subset)

    # Return the information gain as the difference between the total entropy and the weighted entropy
    return total_entropy - weighted_entropy

# 3. Function to get the best feature to split on
def best_feature(dataset):
    features = dataset.columns[:-1]  # All columns except the target column
    best_info_gain = -1
    best_feature = None

    # Calculate the information gain for each feature
    for feature in features:
        info_gain = information_gain(dataset, feature)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature

    return best_feature

# 4. Recursive function to build the decision tree (ID3 with simple pruning)
def id3(dataset, max_depth=None, depth=0):
    # Check for base cases
    if len(dataset.iloc[:, -1].unique()) == 1:  # All examples have the same label
        return dataset.iloc[0, -1]
    if len(dataset.columns) == 1:  # No more features to split on
        return dataset.iloc[:, -1].mode()[0]
    if max_depth is not None and depth >= max_depth:  # Reached max depth
        return dataset.iloc[:, -1].mode()[0]  # Return the most frequent class

    # Choose the best feature to split on
    best = best_feature(dataset)

    # Create the tree as a dictionary
    tree = {best: {}}

    # Split the dataset based on the best feature and build subtrees
    for value in dataset[best].unique():
        subset = dataset[dataset[best] == value]
        tree[best][value] = id3(subset.drop(columns=[best]), max_depth=max_depth, depth=depth+1)

    return tree

# 5. Example of pruning function (simple) - removes nodes that do not help improve accuracy
def prune_tree(tree, test_data):
    if isinstance(tree, dict):
        for key, value in tree.items():
            for sub_key, sub_value in value.items():
                # Recursively prune the subtree
                tree[key][sub_key] = prune_tree(sub_value, test_data)

    # Here you would normally validate the tree and prune it based on test data
    # For simplicity, let's just return the tree unchanged for now
    return tree

# 6. Example Dataset
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot'],
    'Humidity': ['High', 'High', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'High', 'Low'],
    'Windy': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Weak', 'Weak', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No']  # Target variable
}

# 7. Create the dataset
df = pd.DataFrame(data)

# 8. Build the decision tree using ID3
tree = id3(df, max_depth=3)
print("Decision Tree:", tree)

# 9. Prune the decision tree (simple pruning, for now just returns the tree unchanged)
pruned_tree = prune_tree(tree, df)
print("Pruned Decision Tree:", pruned_tree)


Decision Tree: {'Outlook': {'Sunny': {'Temperature': {'Hot': 'No', 'Mild': {'Windy': {'Weak': 'No', 'Strong': 'Yes'}}}}, 'Overcast': 'Yes', 'Rainy': 'Yes'}}
Pruned Decision Tree: {'Outlook': {'Sunny': {'Temperature': {'Hot': 'No', 'Mild': {'Windy': {'Weak': 'No', 'Strong': 'Yes'}}}}, 'Overcast': 'Yes', 'Rainy': 'Yes'}}


In [ ]:
import numpy as np
import pandas as pd

# 1. Helper function to calculate entropy
def entropy(dataset):
    # Count the occurrences of each class label in the dataset
    class_counts = dataset.iloc[:, -1].value_counts()
    # Calculate the entropy of the dataset using the formula
    prob = class_counts / len(dataset)
    return -np.sum(prob * np.log2(prob))

# 2. Function to calculate information gain
def information_gain(dataset, feature):
    # Calculate the entropy of the original dataset
    total_entropy = entropy(dataset)

    # Get the values of the feature and their frequencies
    feature_values = dataset[feature].value_counts()
    weighted_entropy = 0

    # For each feature value, calculate the weighted entropy of the split
    for value, count in feature_values.items():
        subset = dataset[dataset[feature] == value]
        weighted_entropy += (count / len(dataset)) * entropy(subset)

    # Return the information gain as the difference between the total entropy and the weighted entropy
    return total_entropy - weighted_entropy

# 3. Function to get the best feature to split on
def best_feature(dataset):
    features = dataset.columns[:-1]  # All columns except the target column
    best_info_gain = -1
    best_feature = None

    # Calculate the information gain for each feature
    for feature in features:
        info_gain = information_gain(dataset, feature)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature

    return best_feature

# 4. Recursive function to build the decision tree (ID3 with simple pruning)
def id3(dataset, max_depth=None, depth=0):
    # Check for base cases
    if len(dataset.iloc[:, -1].unique()) == 1:  # All examples have the same label
        return dataset.iloc[0, -1]
    if len(dataset.columns) == 1:  # No more features to split on
        return dataset.iloc[:, -1].mode()[0]
    if max_depth is not None and depth >= max_depth:  # Reached max depth
        return dataset.iloc[:, -1].mode()[0]  # Return the most frequent class

    # Choose the best feature to split on
    best = best_feature(dataset)

    # Create the tree as a dictionary
    tree = {best: {}}

    # Split the dataset based on the best feature and build subtrees
    for value in dataset[best].unique():
        subset = dataset[dataset[best] == value]
        tree[best][value] = id3(subset.drop(columns=[best]), max_depth=max_depth, depth=depth+1)

    return tree

# 5. Function to print the decision tree in a tree-like format
def print_tree(tree, depth=0):
    # If the tree is a dictionary, it means it's a decision node
    if isinstance(tree, dict):
        for key, value in tree.items():
            print("  " * depth + str(key))  # Print the feature (node)
            for sub_key, sub_value in value.items():
                print("  " * (depth + 1) + str(sub_key) + ":")
                print_tree(sub_value, depth + 2)  # Recur for the next level
    else:
        print("  " * depth + "-> " + str(tree))  # If it's a leaf, print the class label

# 6. Example Dataset
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot'],
    'Humidity': ['High', 'High', 'High', 'High', 'High', 'Low', 'Low', 'Low', 'High', 'Low'],
    'Windy': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Weak', 'Weak', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No']  # Target variable
}

# 7. Create the dataset
df = pd.DataFrame(data)

# 8. Build the decision tree using ID3
tree = id3(df, max_depth=3)
print("Decision Tree Structure:")
print_tree(tree)


Decision Tree Structure:
Outlook
  Sunny:
    Temperature
      Hot:
        -> No
      Mild:
        Windy
          Weak:
            -> No
          Strong:
            -> Yes
  Overcast:
    -> Yes
  Rainy:
    -> Yes


In [ ]:
import numpy as np
import pandas as pd
from graphviz import Digraph

# 1. Helper function to calculate entropy
def entropy(dataset):
    class_counts = dataset.iloc[:, -1].value_counts()
    prob = class_counts / len(dataset)
    return -np.sum(prob * np.log2(prob))

# 2. Function to calculate information gain
def information_gain(dataset, feature):
    total_entropy = entropy(dataset)
    feature_values = dataset[feature].value_counts()
    weighted_entropy = 0
    for value, count in feature_values.items():
        subset = dataset[dataset[feature] == value]
        weighted_entropy += (count / len(dataset)) * entropy(subset)
    return total_entropy - weighted_entropy

# 3. Function to get the best feature to split on
def best_feature(dataset):
    features = dataset.columns[:-1]
    best_info_gain = -1
    best_feature = None
    for feature in features:
        info_gain = information_gain(dataset, feature)
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_feature = feature
    return best_feature

# 4. Recursive function to build the decision tree (ID3 with simple pruning)
def id3(dataset, max_depth=None, depth=0):
    if len(dataset.iloc[:, -1].unique()) == 1:
        return dataset.iloc[0, -1]
    if len(dataset.columns) == 1:
        return dataset.iloc[:, -1].mode()[0]
    if max_depth is not None and depth >= max_depth:
        return dataset.iloc[:, -1].mode()[0]
    best = best_feature(dataset)
    tree = {best: {}}
    for value in dataset[best].unique():
        subset = dataset[dataset[best] == value]
        tree[best][value] = id3(subset.drop(columns=[best]), max_depth=max_depth, depth=depth+1)
    return tree

# 5. Function to create a graphical decision tree
def create_tree_diagram(tree, dot=None, parent_name="Root", parent_value=""):
    if dot is None:
        dot = Digraph(format="png", engine="dot")

    # Recursively add nodes to the graph
    if isinstance(tree, dict):
        for feature, branches in tree.items():
            feature_name = f"{parent_name}_{feature}"
            dot.node(feature_name, feature)
            dot.edge(parent_name, feature_name, label=parent_value)

            for value, subtree in branches.items():
                value_name = f"{feature_name}_{value}"
                dot.node(value_name, f"{feature}: {value}")
                dot.edge(feature_name, value_name, label=str(value))

                # Recurse for each subtree
                create_tree_diagram(subtree, dot, value_name, str(value))
    else:
        dot.node(parent_name + "_class", f"Class: {tree}")
        dot.edge(parent_name, parent_name + "_class", label="Leaf")

    return dot

# 6. Example Dataset
data = {
    'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy', 'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy'],
    'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool','Mild', 'Cool','Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
    'Windy': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

# 7. Create the dataset
df = pd.DataFrame(data)

# 8. Build the decision tree using ID3
tree = id3(df, max_depth=3)

# 9. Create the decision tree diagram
dot = create_tree_diagram(tree)

# 10. Render and display the tree diagram
dot.render("decision_tree", view=True)  # This will generate a PNG file and open it in the default viewer


'decision_tree.png'